In [3]:
from dotenv import load_dotenv

load_dotenv("../../.env")

True

In [4]:
import requests


def google_search(query, api_key, cse_id, num=10, start=1):
    """
    Google Custom Search API を利用して検索結果を取得する関数
    :param query: 検索キーワード
    :param api_key: Google API キー
    :param cse_id: カスタム検索エンジン ID
    :param num: 1回のリクエストで取得する結果数（最大10件）
    :param start: 検索結果の開始位置（1-indexed）
    :return: API のレスポンス（JSON形式）
    """
    url = "https://www.googleapis.com/customsearch/v1"
    params = {"q": query, "key": api_key, "cx": cse_id, "num": num, "start": start}

    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"エラー: {response.status_code} {response.text}")
        return None


def extract_urls(search_results):
    """
    検索結果から URL リストを抽出する関数
    :param search_results: Google Custom Search API のレスポンス（JSON）
    :return: URL のリスト
    """
    urls = []
    if search_results and "items" in search_results:
        for item in search_results["items"]:
            url = item.get("link")
            if url:
                urls.append(url)
    return urls

In [5]:
import os


# ご自身の API キーとカスタム検索エンジン ID に置き換えてください
api_key = os.getenv("GOOGLE_API_KEY")
cse_id = os.getenv("GOOGLE_CSE_ID")

In [6]:
# 検索したいキーワードを指定
keyword = "ファーストリテイリング 財務諸表"

# 検索を実行（ここでは最初の10件を取得）
search_results = google_search(keyword, api_key, cse_id, num=10, start=1)
urls = extract_urls(search_results)
urls = list(set(urls))

if urls:
    for url_dify in urls:
        print(url_dify)
else:
    print("検索結果が見つかりませんでした。")

https://www.otsu-international.com/financial_statements/07-fastretailing/
https://www.fastretailing.com/jp/ir/library/pdf/tanshin202008_4q.pdf
https://www.fastretailing.com/jp/ir/library/pdf/tanshin202208_4q.pdf
https://www.fastretailing.com/jp/ir/library/pdf/tanshin202308_4q.pdf
https://cpa-apple.com/9309/
http://www.apparel-mag.com/abm/article/business/1848
https://www.fastretailing.com/jp/ir/library/yuho.html
https://zaimani.com/financial-analysis-catalog/9983/
https://www.otsu-international.com/financial_statements/
https://www.fastretailing.com/jp/ir/financial/


In [ ]:
urls[0]

In [72]:
import requests


class DifyClient(object):
    def __init__(self, api_key: str, host: str = "192.168.1.11", user: str = "tkosht"):
        self.api_key = api_key
        self.host = host
        self.user = user

    def run_workflow(self, *args, **kwargs):
        """Dify のワークフローを実行"""


class DifyClientSpider(DifyClient):
    def __init__(self, api_key: str, host: str = "192.168.1.11", user: str = "tkosht"):
        self.api_key = api_key
        self.host = host
        self.user = user

    def run_workflow(self, url: str):
        """
        DIFY のワークフローを実行する関数
        :param url: ワークフローのエンドポイント
        :param api_key: DIFY API キー
        :param user: ユーザ名
        :return: API のレスポンス（JSON形式）
        """
        url_dify = f"http://{self.host}/v1/workflows/run"

        # ヘッダの設定
        headers = {"Authorization": f"Bearer {self.api_key}", "Content-Type": "application/json"}

        # POST データのペイロード
        payload = {
            "inputs": {
                "file_or_url": {
                    "transfer_method": "remote_url",
                    "url": url,
                    "type": "document",
                }
            },
            "response_mode": "blocking",
            "user": self.user,
        }

        # POST リクエストの送信
        response = requests.post(url_dify, headers=headers, json=payload)

        return response

In [ ]:
user = "tkosht"

dcs = DifyClientSpider(os.environ.get("DIFY_API_KEY_MARKDOWN_SPIDER"), host="192.168.1.11", user=user)
dcs

In [ ]:
# url = "https://www.fastretailing.com/jp/ir/financial/"
# url = "https://www.fastretailing.com/jp/ir/financial/summary.html"
url = "https://www.fastretailing.com/jp/ir/library/pdf/tanshin202308_4q.pdf"


response = dcs.run_workflow(url)
print("Status Code:", response.status_code)
response.raise_for_status()

In [ ]:
response.text

In [ ]:
import json


res = json.loads(response.text)
res

In [ ]:
from IPython.display import Markdown, display, HTML

md = res["data"]["outputs"]["text"]
display(Markdown(md))

In [75]:
class DifyClientSummary(DifyClient):
    def __init__(self, api_key: str, host: str = "192.168.1.11", user: str = "tkosht"):
        self.api_key = api_key
        self.host = host
        self.user = user

    def run_workflow(self, query: str, text: str):
        """
        DIFY のワークフローを実行する関数
        :param url: ワークフローのエンドポイント
        :param api_key: DIFY API キー
        :param user: ユーザ名
        :return: API のレスポンス（JSON形式）
        """
        url_dify = f"http://{self.host}/v1/workflows/run"

        # ヘッダの設定
        headers = {"Authorization": f"Bearer {self.api_key}", "Content-Type": "application/json"}

        # POST データのペイロード
        payload = {
            "inputs": {
                "query": query,
                "text": text,
            },
            "response_mode": "blocking",
            "user": self.user,
        }

        # POST リクエストの送信
        response = requests.post(url_dify, headers=headers, json=payload)

        return response

In [ ]:
dsm = DifyClientSummary(os.environ.get("DIFY_API_KEY_SUMMARY_PAPER"), host="192.168.1.11", user=user)
dsm

In [ ]:
response = dsm.run_workflow(query="財務諸表の分析を精緻にしたい", text=md)
print("Status Code:", response.status_code)
response.raise_for_status()

In [ ]:
res = json.loads(response.text)
res

In [ ]:
display(Markdown(res["data"]["outputs"]["text"]))

In [ ]:
len(res["data"]["outputs"]["text"])